## Import

In [1]:
# import sys
# sys.path.append(r'C:\Users\hyeJunoh\Desktop\인턴폴더\연구')

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable

import torchvision
import torchvision.transforms as transforms
from torchvision import models, datasets

from sklearn.metrics import f1_score, confusion_matrix
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE

import os
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import random
import h5py
import glob
from pathlib import Path
import pickle
import argparse
import time


import seaborn as sn
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings(action='ignore')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


c:\Users\SHRM_DL\.conda\envs\jun\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\SHRM_DL\.conda\envs\jun\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\SHRM_DL\.conda\envs\jun\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
from fft import FFT
from loader import dataloader
from train_teacher import teacher_train
from test_teacher import testing_teacher
from model.teacher_model import teacher_encoder, teacher_classifier
from model.attention import SpatialAttention, MultiviewCrossAttention
from model.cnn import BasicBlock,GlobalAvgPool1D


In [3]:
CFG = {
    'EPOCHS': 80,
    'LEARNING_RATE':1e-2,
    'BATCH_SIZE':256,
    'SEED':41
}

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED'])

## Data Load

In [4]:
source1_train_dataloader,source1_valid_dataloader,source1_test_dataloader,source2_train_dataloader,source2_valid_dataloader,source2_test_dataloader,target_train_dataloader,target_valid_dataloader,target_test_dataloaders=dataloader()

# Teacher model

In [6]:
def teacher_enc(pretrained=False, **kwargs):
    model = teacher_encoder(BasicBlock, [2, 2, 2, 2], **kwargs)
    return model## Run (Teacher Model)
def teacher_clas(pretrained=False, **kwargs):
    model = teacher_classifier(BasicBlock, [2, 2, 2, 2], **kwargs)
    return model## Run (Teacher Model)
teacher_encoder = teacher_enc().to(device)
teacher_classifier = teacher_clas().to(device)
Spectral_Attention_cur = SpatialAttention(in_channels=32, reduction_ratio=16, version='cur').to(device)
Spectral_Attention_vib = SpatialAttention(in_channels=32, reduction_ratio=16, version='vib').to(device)
MultiviewCrossAttention = CrossAttention(input_size=4096, version='default').to(device)
temporal_attention_c = SpatialAttention(in_channels=32, reduction_ratio=16, version='cur_2').to(device)
temporal_attention_v = SpatialAttention(in_channels=32, reduction_ratio=16, version='vib_2').to(device)

In [7]:
optimizer = torch.optim.Adam([
                        {'params': teacher_encoder.parameters()},
                        {'params': Spectral_Attention_cur.parameters()},
                        {'params': Spectral_Attention_vib.parameters()},
                        {'params': MultiviewCrossAttention.parameters()},
                        {'params': temporal_attention_c.parameters()},
                        {'params': temporal_attention_v.parameters()},
                        {'params': teacher_classifier.parameters()},
                        ], lr= 0.0001)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=15, threshold_mode='abs',min_lr=1e-8, verbose=True)

In [8]:
best_teacher_encoder,best_temporal_attention_cur,best_temporal_attention_vib,best_spectral_attention_cur,best_spectral_attention_vib,best_multiview_CrossAttention,best_teacher_classifier=teacher_train(teacher_encoder,temporal_attention_c,temporal_attention_v,Spectral_Attention_cur,Spectral_Attention_vib,MultiviewCrossAttention,teacher_classifier, optimizer, source1_train_dataloader,source2_train_dataloader, source1_valid_dataloader,source2_valid_dataloader, scheduler, device,epochs=30)

epoch:  0
source 1 train


100%|██████████| 1394/1394 [00:09<00:00, 140.84it/s]


Epoch [0], Train Loss : [0.14672] Val Loss : [1.25505] Val ACC Score : [50.07174]
source 2 train


100%|██████████| 1394/1394 [00:09<00:00, 140.24it/s]


Epoch [0], Train Loss : [0.12311] Val Loss : [0.84279] Val ACC Score : [80.05739]
epoch:  1
source 1 train


...

100%|██████████| 1394/1394 [00:10<00:00, 138.97it/s]


Epoch [23], Train Loss : [0.15085] Val Loss : [0.24799] Val ACC Score : [99.13917]
source 2 train


100%|██████████| 1394/1394 [00:10<00:00, 139.33it/s]


Epoch [23], Train Loss : [0.14121] Val Loss : [0.22014] Val ACC Score : [100.00000]
epoch:  24
source 1 train


  3%|▎         | 45/1394 [00:00<00:09, 139.54it/s]

In [ ]:
torch.save(best_teacher_encoder.state_dict(),open("../weights/best_teacher_encoder.pth","wb"))
torch.save(best_temporal_attention_cur.state_dict(),open("../weights/best_temporal_attention_cur.pth","wb"))
torch.save(best_temporal_attention_vib.state_dict(),open("../weights/best_temporal_attention_vib.pth","wb"))
torch.save(best_spectral_attention_cur.state_dict(),open("../weights/best_spectral_attention_cur.pth","wb"))
torch.save(best_spectral_attention_vib.state_dict(),open("../weights/best_spectral_attention_vib.pth","wb"))
torch.save(best_multiview_CrossAttention.state_dict(),open("../weights/best_multiview_CrossAttention.pth","wb"))
torch.save(best_teacher_classifier.state_dict(),open("../weights/best_teacher_classifier.pth","wb"))

In [ ]:
teacher_test_loss, teacher_test_acc=testing_teacher(best_teacher_encoder,best_temporal_attention_cur,best_temporal_attention_vib,best_spectral_attention_cur,best_spectral_attention_vib,best_multiview_CrossAttention,best_teacher_classifier,source1_test_dataloader)
print("test teacher loss: ",round(sum(teacher_test_loss)/len(teacher_test_loss),3), "test teacher accuracy: ", teacher_test_acc)

teacher_test_loss, teacher_test_acc=testing_teacher(best_teacher_encoder,best_temporal_attention_cur,best_temporal_attention_vib,best_spectral_attention_cur,best_spectral_attention_vib,best_multiview_CrossAttention,best_teacher_classifier,source2_test_dataloader)
print("test teacher loss: ",round(sum(teacher_test_loss)/len(teacher_test_loss),3), "test teacher accuracy: ", teacher_test_acc)

100%|██████████| 100/100 [00:00<00:00, 141.72it/s]


test teacher loss:  1.609 test teacher accuracy:  23.0


100%|██████████| 100/100 [00:00<00:00, 141.22it/s]

test teacher loss:  1.611 test teacher accuracy:  13.0
